# Functions (advanced)

[TOC]

We are going to cover the following topics:

- Decorators
- Lambdas
- Arguments and object's mutability
- Generators

## Recap

Before starting our deep dive on functions, we must revise quickly two important concepts. Have a look at the [Functions](./functions.ipynb#The-scope-of-a-function) notebook for more detail.

1. Scopes and namespaces
2. Functions are objects

### Functions are objects

As an example, suppose that we want to create a "password checker", that is, a function that can verify if an input password complies with some rules (e.g., minumum length, a given number of special characters). We could create a function with the following signature:

```python
def check_password(
    password: str,
    min_length: int,
    min_uppercase: int,
    min_punctuation:
    int, min_digits: int
    ) -> bool:
    """Check a given password again"""
```

The obvious problem is that different passwords might adhere to different rules. We would need to specify those rules for every password that should comply to those, and that's tedious!

We can instead define a so-called **higher-order function** (see [Functional programming](./functional_programming.ipynb)): a function that returns another function.
It does **not** call that function, just returns it.

```python
from typing import Callable

def check_password_factory(
    password: str,
    min_length: int,
    min_uppercase: int,
    min_punctuation:
    int, min_digits: int
    ) -> Callable:
    """Our password checker factor"""

    def check_password(password: str) -> bool:
        """Password checker function"""
        # our password checking logic
        # ...
        return # True or False

    return check_password
```

### Scopes and namespaces

Python's variables are just names (i.e., labels) that we can **bind** to objects. Each variable is simply telling Python where to look in our computer's memory to retrieve some data. These bindings are **not universal**: some of them exist only in specific parts of our code.

> The portion of code where a name binding is defined is called **lexical scope** (or just "scope"). The bindings are stored in a scope's **namespace**

We always have the following scopes:

1. `built-in` scope
2. `global` (or module) scope

We also have the `local` scope that's created we are **calling** a function.
The local scope associated to any called function is **destroyed** after the function has done its job. Also the namespace associated with it will be gone.

When Python needs to retrieve which object is referenced by a given name, it always starts from the current scope (the `local` one if we are inside a function's body). If a name binding is not found there, it searches in the scope immediately up in the hierarchy.

> **LEGB rule**: **L**ocal → **E**nclosing → **G**lobal → **B**uilt-in

When Python encounters a function **definition** (i.e., at compile-time), it does two things:

1. Scans for any variables that have values **assigned** anywhere in the function. By default, names that are assigned are **local** unless we are explicitly saying that they should not with the `global` keyword.
2. Variables that are **referenced** but **not assigned** a value anywhere in the function will **not be local**. When we are calling the function (i.e., run-time), Python will look for them in the **enclosing scope**.

Examples:

```python
var = 10   # global (or module) scope

def func_1():
    print(var)   # var is referenced but not assigned. At compile-time is "non-local"

def func_2()
    var = 100    # var is assigned. At compile-time will be placed in the "local" scope

def func_3():
    global var
    var = 1000   # var is assigned, so it should be local. But it's also declared to be "global" with the keyword above

def func_4():
    print(var)
    var = 100    # what happens here?
```

A function gets its local scope upon calling. Since we can have function definitions inside of other functions, there can be **nested scopes**. This is where the `nonlocal` keyword becomes useful or even needed.

> The `nonlocal` keyword is used to declare that a variable is not local to the current function but is defined in the **nearest enclosing scope** that is **not global**. It allows you to access and modify variables in the outer (non-global) scope from within an inner function.

An example:

In [ ]:
def outer_function():
    x = "outer"

    # prints "outer"
    print("Inside outer_function:", x)

    def inner_function():
        nonlocal x
        x = "inner"
        # prints "inner"
        print("Inside inner_function:", x)

    inner_function()
    # prints "inner" again, because we modified `x` from a nested scope
    print("Inside outer_function:", x)

outer_function()

Two important notes about the `nonlocal` keyword:

1. Python will search for a `nonlocal` name in the **enclosing local scopes** until it first encounters the specified variable.
2. **Only** local scopes are searched, never the global one.

## Closures

Let's consider the following code:

In [ ]:
def outer():
    lang = "Python"

    def inner():
        print(f"{lang} rocks!")

    inner()

outer()

Here the `lang` variable is **non-local** to `inner()` because it's only referenced. `lang` is also called **free variable**.

> A **free variable** is a variable referenced locally but defined in the enclosing scope.

Also, `lang` and `inner()` both belongs to the local scope of `outer()`. Since this bound is particularly special, it has a special name: it's called a **closure**.
The name "closure" come from the function `inner()` _enclosing_ its free variable `lang`.

Let's make a small adjustment that will change a lot of things:

In [ ]:
def outer():
    lang = "Python"

    def inner():
        print(f"{lang} rocks!")

    return inner

outer()

We turned `outer()` into an higher-order function that does not return a simple function, but a closure (`inner()` + the free variable).

Since functions are objects, we can assign that to a name:

In [ ]:
fn = outer()

And then call that function as any other function:

In [ ]:
fn()

But wait a second! How's that possible? 🤔

`fn()` is called **after** `outer()` has run: it runs when we are assigning the name `fn` to the result of calling `outer()`.
If the local scope of a function is destroyed after the function has run, how can `fn` know that `lang = "Python"`?

That's because Pyhon realized that we created a closure, and it's doing something unusual.

If we look once again to the example above, we see that the name `lang` is **shared** between the local scope of `outer()` and the `print` statement inside `inner()`.
When Python sees this, it does something different: it creates an **intermediary** object – called a _cell object_ – that only contains a memory address.
A memory address of what? Of whatever object (i.e. data) is assigned to `lang`, the free variable.

![](./images/cell_object.png)

We can see all that by inspecting some _hidden_ attributes of `fn`:

In [ ]:
fn.__code__.co_freevars

In [ ]:
fn.__closure__

We can see now the reason why we can call `fn()` and see the string "Python rocks!" printed although the variable `lang` is now out of scope (it's been destroyed).
There is another reference to the cell object, that from the closure created by `inner()` plus the free variable.
When running `outer()`, `inner()` is **not called**, and Python still knows how to retrieve the value of the string object.

### Modifying the free variable

### Multiple closures

### Closures can be tricky

### Nested closures

## Decorators

## Lambdas

## Objects mutability

## Generators

## Exercises

In [ ]:
%reload_ext tutorial.tests.testsuite

### Once per minute

Create a decorator called `once` that restricts a function to run at most **once every 15 seconds**.
If you try to invoke the function too soon, the decorator should raise an exception called `TooSoonError` which tells you how long you need to wait before running your function again.

For example, the following code:

```python
import time

@once
def hello(name):
    return f"Hello, {name}!"

for i in range(30):
    print(i)
    try:
        time.sleep(3)
        print(hello(f"attempt #{i}"))
    except TooSoonError as err:
        print(f"Too soon: {err}")
```

Should print something like:

```
0
Hello, attempt #0
1
Too soon: Wait another 12.00 seconds
2
Too soon: Wait another 8.99 seconds
3
Too soon: Wait another 5.98 seconds
4
Too soon: Wait another 2.98 seconds
5
Hello, attempt #5
6
Too soon: Wait another 12.00 seconds
```

<div class="alert alert-block alert-warning">
    <h4><b>Note</b></h4> The decorator should handle <strong>any kind</strong> of function, i.e., it should not care about the kind or number of parameters the function accepts.
</div>

In [ ]:
%%ipytest

def solution_once(func):
    """Restrict running a function at most once every 15 seconds"""
    def wrapper(*args, **kwargs):
        """Your wrapper"""
        return func(*args, **kwargs)

    return wrapper

### Password checker `{functions as objects}`

Create a "password checker factory"

### String range `{generators}`

Create a generator function that emulates the the built-in `range`, but for characters

### Read `n` lines `{generators}`

Create a generator function that reads in the content of a file by `n` lines at a time